In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
!pip install tensorflow-model-optimization
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.models import load_model
from google.colab import files
import shutil
import gc
from keras.datasets import cifar10
from tensorflow.keras import layers
import shutil
from google.colab import drive
drive.mount('/content/drive')

np.random.seed(42)
tf.random.set_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 2.1 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
num_classes = 10
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



170498071/170498071 [==============================] - 3s 0us/step
x_train shape: (50000, 32, 32, 3, 1)
50000 train samples
10000 test samples


In [ ]:
model_name = 'mobilenet'
dataset = 'CIFAR10'

---

# Distillation

In [ ]:
def train_custom(model, train_data, train_labels, test_data, test_labels):
    initailsoftmax = model.student.predict(test_data)
    model.student.save_weights(init_weights)
    model.student.save(init_model)
    initaildf = pd.DataFrame(initailsoftmax)
    filename = filepath + '0_softmax.csv'
    initaildf.to_csv(filename,index=False)
    epoch = 1
    while epoch < 26:
        print(f"Epoch {epoch}:")

        # Training on one epoch
        model.fit(train_data, train_labels, epochs=1, batch_size=64, verbose=1)

        # Evaluate on the test dataset
        results = model.student.predict(test_data)
        softmax_df = pd.DataFrame(results)
        filename = filepath+str(epoch)+'_softmax.csv'
        softmax_df.to_csv(filename,index=False)
        loss,accuracy = model.student.evaluate(test_data,test_labels)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

        if epoch ==25:
          model.student.save_weights(final_weights)
          model.student.save(final_model)
        epoch += 1

# TO DO EVALUATE FUNCTION


In [ ]:
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha,
        temperature,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )

        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)
        print(y_prediction)
        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [ ]:
teacher_model_path = '/content/base/mobilenetCIFAR10_final.tf'
techer_model = load_model(teacher_model_path)

# Student 0.1 Alpha

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/mobilenetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 4s 5ms/step


Epoch 1:
313/313 [==============================] - 2s 6ms/step - loss: 1.9380 - accuracy: 0.2846
Test Loss: 1.9379723072052002, Test Accuracy: 0.28459998965263367
Epoch 2:
313/313 [==============================] - 2s 6ms/step - loss: 1.7040 - accuracy: 0.3761
Test Loss: 1.7039968967437744, Test Accuracy: 0.37610000371932983
Epoch 3:
313/313 [==============================] - 2s 7ms/step - loss: 1.6876 - accuracy: 0.4175
Test Loss: 1.6876131296157837, Test Accuracy: 0.41749998927116394
Epoch 4:
313/313 [==============================] - 2s 8ms/step - loss: 1.5683 - accuracy: 0.4705
Test Loss: 1.5683164596557617, Test Accuracy: 0.47049999237060547
Epoch 5:
313/313 [==============================] - 3s 8ms/step - loss: 1.5251 - accuracy: 0.4821
Test Loss: 1.5251171588897705, Test Accuracy: 0.4821000099182129
Epoch 6:
313/313 [==============================] - 2s 5ms/step - loss: 1.6753 - accuracy: 0.4855
Test Loss: 1.6753177642822266, Test Accuracy: 0.4855000078678131
Epoch 7:
313/313 [

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same", 'zip', "student_same")
colab_link = "/content/student_same.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C10/student_same.zip'

# Student 0.1 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/mobilenetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 2s 4ms/step
Epoch 1:
313/313 [==============================] - 3s 7ms/step - loss: 1.9187 - accuracy: 0.2665
Test Loss: 1.9187183380126953, Test Accuracy: 0.26649999618530273
Epoch 2:
313/313 [==============================] - 2s 8ms/step - loss: 1.6745 - accuracy: 0.4062
Test Loss: 1.6744861602783203, Test Accuracy: 0.40619999170303345
Epoch 3:
313/313 [==============================] - 2s 6ms/step - loss: 1.5435 - accuracy: 0.4521
Test Loss: 1.5435075759887695, Test Accuracy: 0.45210000872612
Epoch 4:
313/313 [==============================] - 2s 7ms/step - loss: 1.4928 - accuracy: 0.4905
Test Loss: 1.492781162261963, Test Accuracy: 0.49050000309944153
Epoch 5:
313/313 [==============================] - 3s 9ms/step - loss: 1.3773 - accuracy: 0.5267
Test Loss: 1.3772764205932617, Test Accuracy: 0.5267000198364258
Epoch 6:
313/313 [==============================] - 2s 6ms/step - loss: 1.4299 - accuracy: 0.5245
Test Loss: 1.4298558235168457, T

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_2", 'zip', "student_same_2")
colab_link = "/content/student_same_2.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C10/student_same_2.zip'

# student 0.1 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/mobilenetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 2s 6ms/step
Epoch 1:
313/313 [==============================] - 2s 6ms/step - loss: 1.9059 - accuracy: 0.2815
Test Loss: 1.905892014503479, Test Accuracy: 0.2815000116825104
Epoch 2:
313/313 [==============================] - 2s 6ms/step - loss: 1.6802 - accuracy: 0.3741
Test Loss: 1.6801791191101074, Test Accuracy: 0.374099999666214
Epoch 3:
313/313 [==============================] - 2s 7ms/step - loss: 1.5278 - accuracy: 0.4551
Test Loss: 1.5277901887893677, Test Accuracy: 0.45509999990463257
Epoch 4:
313/313 [==============================] - 2s 8ms/step - loss: 1.4557 - accuracy: 0.4990
Test Loss: 1.4557040929794312, Test Accuracy: 0.49900001287460327
Epoch 5:
313/313 [==============================] - 3s 8ms/step - loss: 1.4540 - accuracy: 0.5066
Test Loss: 1.453996181488037, Test Accuracy: 0.506600022315979
Epoch 6:
313/313 [==============================] - 2s 6ms/step - loss: 1.4161 - accuracy: 0.5290
Test Loss: 1.4161456823349, Test A

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_3", 'zip', "student_same_3")
colab_link = "/content/student_same_3.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C10/student_same_3.zip'

# Student same Alpha 0.5

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/mobilenetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 3s 6ms/step
Epoch 1:
313/313 [==============================] - 2s 6ms/step - loss: 1.9180 - accuracy: 0.2790
Test Loss: 1.918038249015808, Test Accuracy: 0.27900001406669617
Epoch 2:
313/313 [==============================] - 2s 6ms/step - loss: 1.6239 - accuracy: 0.4003
Test Loss: 1.6239248514175415, Test Accuracy: 0.400299996137619
Epoch 3:
313/313 [==============================] - 3s 8ms/step - loss: 1.6034 - accuracy: 0.4287
Test Loss: 1.6033889055252075, Test Accuracy: 0.4287000000476837
Epoch 4:
313/313 [==============================] - 3s 9ms/step - loss: 1.4407 - accuracy: 0.4857
Test Loss: 1.4406628608703613, Test Accuracy: 0.48570001125335693
Epoch 5:
313/313 [==============================] - 3s 9ms/step - loss: 1.4146 - accuracy: 0.5082
Test Loss: 1.4145581722259521, Test Accuracy: 0.5081999897956848
Epoch 6:
313/313 [==============================] - 3s 8ms/step - loss: 1.3940 - accuracy: 0.5274
Test Loss: 1.3940398693084717, T

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50", 'zip', "student_same_50")
colab_link = "/content/student_same_50.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C10/student_same_50.zip'

# student 0.5 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/mobilenetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 3s 6ms/step
Epoch 1:
313/313 [==============================] - 2s 6ms/step - loss: 1.9041 - accuracy: 0.2738
Test Loss: 1.9040714502334595, Test Accuracy: 0.27379998564720154
Epoch 2:
313/313 [==============================] - 2s 7ms/step - loss: 1.7194 - accuracy: 0.4032
Test Loss: 1.719398856163025, Test Accuracy: 0.4032000005245209
Epoch 3:
313/313 [==============================] - 2s 6ms/step - loss: 1.5526 - accuracy: 0.4515
Test Loss: 1.552613615989685, Test Accuracy: 0.4514999985694885
Epoch 4:
313/313 [==============================] - 2s 6ms/step - loss: 1.5174 - accuracy: 0.4698
Test Loss: 1.51738440990448, Test Accuracy: 0.4697999954223633
Epoch 5:
313/313 [==============================] - 3s 8ms/step - loss: 1.3709 - accuracy: 0.5220
Test Loss: 1.3709404468536377, Test Accuracy: 0.5220000147819519
Epoch 6:
313/313 [==============================] - 3s 8ms/step - loss: 1.4143 - accuracy: 0.5255
Test Loss: 1.414292335510254, Test 

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50_2", 'zip', "student_same_50_2")
colab_link = "/content/student_same_50_2.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C10/student_same_50_2.zip'

# Student 0.5 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/mobilenetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 2s 6ms/step
Epoch 1:
313/313 [==============================] - 3s 6ms/step - loss: 1.9500 - accuracy: 0.2703
Test Loss: 1.949955701828003, Test Accuracy: 0.2703000009059906
Epoch 2:
313/313 [==============================] - 2s 6ms/step - loss: 1.7112 - accuracy: 0.3860
Test Loss: 1.7112284898757935, Test Accuracy: 0.38600000739097595
Epoch 3:
313/313 [==============================] - 3s 9ms/step - loss: 1.6263 - accuracy: 0.4297
Test Loss: 1.62628173828125, Test Accuracy: 0.42969998717308044
Epoch 4:
313/313 [==============================] - 2s 6ms/step - loss: 1.4635 - accuracy: 0.4908
Test Loss: 1.463485598564148, Test Accuracy: 0.49079999327659607
Epoch 5:
313/313 [==============================] - 2s 6ms/step - loss: 1.5080 - accuracy: 0.5070
Test Loss: 1.5080198049545288, Test Accuracy: 0.5070000290870667
Epoch 6:
313/313 [==============================] - 2s 6ms/step - loss: 1.4716 - accuracy: 0.5236
Test Loss: 1.4716393947601318, Te

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50_3", 'zip', "student_same_50_3")
colab_link = "/content/student_same_50_3.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C10/student_same_50_3.zip'

# Student same Alpha 0.9

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/mobilenetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 2s 4ms/step
Epoch 1:
313/313 [==============================] - 3s 6ms/step - loss: 1.9641 - accuracy: 0.2735
Test Loss: 1.9640673398971558, Test Accuracy: 0.273499995470047
Epoch 2:
313/313 [==============================] - 2s 6ms/step - loss: 1.6053 - accuracy: 0.4138
Test Loss: 1.60530686378479, Test Accuracy: 0.4138000011444092
Epoch 3:
313/313 [==============================] - 2s 6ms/step - loss: 1.6324 - accuracy: 0.4287
Test Loss: 1.6324031352996826, Test Accuracy: 0.4287000000476837
Epoch 4:
313/313 [==============================] - 3s 9ms/step - loss: 1.4451 - accuracy: 0.4880
Test Loss: 1.445094347000122, Test Accuracy: 0.4880000054836273
Epoch 5:
313/313 [==============================] - 2s 6ms/step - loss: 1.3617 - accuracy: 0.5246
Test Loss: 1.3616504669189453, Test Accuracy: 0.5246000289916992
Epoch 6:
313/313 [==============================] - 2s 6ms/step - loss: 1.4928 - accuracy: 0.5055
Test Loss: 1.492775559425354, Test A

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90", 'zip', "student_same_90")
colab_link = "/content/student_same_90.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C10/student_same_90.zip'

# Student 0.9 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/mobilenetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 2s 4ms/step
Epoch 1:
313/313 [==============================] - 3s 7ms/step - loss: 1.9130 - accuracy: 0.2919
Test Loss: 1.9129806756973267, Test Accuracy: 0.29190000891685486
Epoch 2:
313/313 [==============================] - 2s 6ms/step - loss: 1.6484 - accuracy: 0.3993
Test Loss: 1.648423671722412, Test Accuracy: 0.3993000090122223
Epoch 3:
313/313 [==============================] - 2s 6ms/step - loss: 1.5262 - accuracy: 0.4652
Test Loss: 1.5262436866760254, Test Accuracy: 0.4652000069618225
Epoch 4:
313/313 [==============================] - 2s 8ms/step - loss: 1.4724 - accuracy: 0.4853
Test Loss: 1.4723732471466064, Test Accuracy: 0.4853000044822693
Epoch 5:
313/313 [==============================] - 3s 8ms/step - loss: 1.4036 - accuracy: 0.5099
Test Loss: 1.4035625457763672, Test Accuracy: 0.5098999738693237
Epoch 6:
313/313 [==============================] - 2s 6ms/step - loss: 1.4957 - accuracy: 0.5016
Test Loss: 1.4957233667373657, T

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90_2", 'zip', "student_same_90_2")
colab_link = "/content/student_same_90_2.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C10/student_same_90_2.zip'

# Student 0.9 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/mobilenetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 2s 4ms/step
Epoch 1:
313/313 [==============================] - 2s 6ms/step - loss: 1.9731 - accuracy: 0.2616
Test Loss: 1.9731343984603882, Test Accuracy: 0.26159998774528503
Epoch 2:
313/313 [==============================] - 2s 6ms/step - loss: 1.6402 - accuracy: 0.4013
Test Loss: 1.6401948928833008, Test Accuracy: 0.40130001306533813
Epoch 3:
313/313 [==============================] - 2s 6ms/step - loss: 1.5711 - accuracy: 0.4468
Test Loss: 1.5711417198181152, Test Accuracy: 0.44679999351501465
Epoch 4:
313/313 [==============================] - 2s 7ms/step - loss: 1.5558 - accuracy: 0.4701
Test Loss: 1.5558316707611084, Test Accuracy: 0.4700999855995178
Epoch 5:
313/313 [==============================] - 2s 6ms/step - loss: 1.4545 - accuracy: 0.5000
Test Loss: 1.4545478820800781, Test Accuracy: 0.5
Epoch 6:
313/313 [==============================] - 2s 6ms/step - loss: 1.5771 - accuracy: 0.4994
Test Loss: 1.577103853225708, Test Accuracy:

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90_3", 'zip', "student_same_90_3")
colab_link = "/content/student_same_90_3.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C10/student_same_90_3.zip'

---